# イベント画像を OpenAI で解析する

---

## Guide page
このサンプルを実行する場合は、必ず以下の**ガイドページ**を確認してください。
- https://users.soracom.io/ja-jp/docs/soracom-cloud-camera-services/api-examples-analyze-event-image-with-openai/


## License

We are using the following libraries.
- **openai / openai-python**
  - https://github.com/openai/openai-python

The license complies with the library's license.
- **Apache License 2.0**
  - https://github.com/openai/openai-python/blob/main/LICENSE

In [ ]:
# @title ステップ 1: OpenAI のライブラリをインストールする

# --------------- Execution ---------------

# When executing 'pip install', an error is displayed due to the absence of the following dependent libraries, but it is commented out as it does not affect the execution.
# !pip install --upgrade cohere
# !pip install --upgrade tiktoken

!pip install --upgrade openai

In [ ]:
# @title ステップ 2: サンプルコードの実行に必要なライブラリや定数を設定する

# --------------- Definition ---------------

from urllib.parse import urljoin, urlparse, quote, unquote
from datetime import datetime, timedelta, timezone
from typing import Final, Union
from google.colab import files

import ipywidgets as widgets
import requests
import shutil
import glob
import copy
import json
import sys
import os


# API Execution Result Class
class APIResult():
    response: dict
    additions: dict

    def __init_vars(self):
        self.response = None
        self.additions = None

    def __init__(self, response: dict, additions: dict):
        self.__init_vars()
        self.response = response
        self.additions = additions

    def get_display_response(self) -> str:
        if self.response != None:
            return json.dumps(self.response, indent=4, ensure_ascii=False)
        return ''

    def get_display_additions(self) -> str:
        if self.additions != None:
            return json.dumps(self.additions, indent=4, ensure_ascii=False)
        return ''

    def check_success(self) -> bool:
        if self.additions != None:
            return True if self.additions['status_code'] == 200 else False
        return False


# Get Contents Execution Result Class
class ContentsResult():
    content: bytes
    additions: dict

    def __init_vars(self):
        self.content = None
        self.additions = None

    def __init__(self, content: bytes, additions: dict):
        self.__init_vars()
        self.content = content
        self.additions = additions

    def get_display_additions(self) -> str:
        if self.additions != None:
            return json.dumps(self.additions, indent=4, ensure_ascii=False)
        return ''

    def check_success(self) -> bool:
        if self.additions != None:
            return True if self.additions['status_code'] == 200 else False
        return False


# Export Time Management Class
class ExportTimeInfo():
    time_from: datetime
    time_to: datetime

    def init_vars(self):
        self.time_from = None
        self.time_to = None

    def valid_vars(self) -> bool:
        return True if self.time_from != None and self.time_to != None else False

    def __init__(self, time_from: datetime, time_to: datetime):
        self.init_vars()
        self.time_from = time_from
        self.time_to = time_to

    def __get_range_limit(self) -> int:
        # As of June 2023, there is a limit of 15 minutes per video export.
        # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceStreamingVideo
        EXPORT_RANGE_LIMIT_SECONDS: Final = 900
        return EXPORT_RANGE_LIMIT_SECONDS

    def get_from_ms(self) -> Union[int, None]:
        if self.time_from != None:
            return int(self.time_from.timestamp() * 1000)
        return None

    def get_to_ms(self) -> Union[int, None]:
        if self.time_to != None:
            return int(self.time_to.timestamp() * 1000)
        return None

    def get_export_sec(self) -> Union[int, None]:
        if self.valid_vars():
            return (self.time_to - self.time_from).total_seconds()
        return None

    def check_time_limits(self, range_limit: bool = True) -> bool:
        if not self.valid_vars():
            return False

        jst_now = datetime.now(JST)

        # Future time cannot be specified.
        if jst_now.timestamp() < self.time_from.timestamp() or jst_now.timestamp() < self.time_to.timestamp():
            print()
            print(
                '➕➕➕ ⛔ A future date and time has been specified. Please confirm the date and time again. ➕➕➕')
            return False

        # If there is no difference between start and end time
        if self.get_export_sec() <= 0:
            print()
            print(
                '➕➕➕ ⛔ The date and time specified is incorrect. Please check the date and time specified. ➕➕➕')
            return False

        # Factoring in API unit output limits
        if range_limit:
            if self.get_export_sec() > self.__get_range_limit():
                print()
                print(
                    '➕➕➕ ⛔ The specified date and time range exceeds {} seconds. Please specify within the box.➕➕➕'.format(self.__get_range_limit()))
                return False

        return True

    def create_time_list(self) -> list:
        time_list = list()

        if not self.valid_vars():
            return time_list

        start = self.time_from
        end = self.time_to

        if not self.check_time_limits(False):
            return time_list

        # Use as is if not caught by the output limit of the export unit.
        if (end - start).total_seconds() <= self.__get_range_limit():
            time_list.append(ExportTimeInfo(start, end))
            return time_list

        # Divide by output unit.
        while True:
            # Add Output Units Only
            end = start + timedelta(seconds=self.__get_range_limit())

            # The calculated time is the future time and should be the specified time.
            if (self.time_to - end).total_seconds() <= 0:
                time_list.append(ExportTimeInfo(start, self.time_to))
                break

            time_list.append(ExportTimeInfo(start, end))
            start = end

        return time_list


# Definition of constants

# Japan Time Zone
JST: Final = timezone(timedelta(hours=+9), 'JST')

# Working directory when creating a ZIP file
ZIP_WORK_DIR: Final = './__zip_work'


# Process start message display
def print_start_msg() -> None:
    print()
    print('# ===== 🚩 Start processing ===== ', datetime.now(JST))

# Process end message display
def print_end_msg() -> None:
    print()
    print('# ===== 🍵 End processing ===== ', datetime.now(JST))

# Combine URL Strings
def merge_url(base: str, url: str) -> str:
    print()
    print('# 🧰 Combine URL Strings')
    print('# base = ', base)
    print('# url = ', url)

    parsed = urlparse(base)
    if parsed.path != '':
        if not parsed.path.endswith('/'):
            # add '/' because it is missing at the end
            base += '/'

    return urljoin(base, url.replace('/', '', 1))

# Send HTTP POST request
def send_post_request(url: str, headers: dict, data: dict, log: bool = False) -> APIResult:
    print()
    print('# 📡 Send HTTP POST request')
    print('# url = ', url)

    if log:
        print('# headers = ', headers)
        print('# data = ', data)

    # Send Request
    resp = requests.post(url, headers=headers, data=json.dumps(data))

    # Display request datetime in JST
    # Mon, 06 Feb 2023 02:03:09 GMT
    req_time = datetime.strptime(
        resp.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').astimezone(JST)
    print('# ⏰ Request datetime = ', req_time,
          int(req_time.timestamp() * 1000))

    # Preparation for results return
    additions = dict()
    additions['status_code'] = resp.status_code
    additions['headers'] = resp.headers

    if resp.status_code != 200:
        print()
        print('# ❌ POST request failed')
        print('# Status Code = ', resp.status_code)
        print('# Raw Messages = ', resp.json())
        additions['raw_messages'] = resp.json

    return APIResult(resp.json(), additions)

# Send HTTP GET request
def send_get_request(url: str, headers: dict, params: dict = None, log: bool = False) -> APIResult:
    print()
    print('# 📡 Send HTTP GET request')
    print('# url = ', url)

    if log:
        print('# headers = ', headers)
    if params != None:
        print('# params = ', params)

    # Send Request
    resp = requests.get(url, headers=headers, params=params)

    # Display request datetime in JST
    # Mon, 06 Feb 2023 02:03:09 GMT
    req_time = datetime.strptime(
        resp.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').astimezone(JST)
    print('# ⏰ Request datetime = ', req_time,
          int(req_time.timestamp() * 1000))

    # Preparation for results return
    additions = dict()
    additions['status_code'] = resp.status_code
    additions['headers'] = resp.headers

    if resp.status_code != 200:
        print()
        print('# ❌ GET request failed')
        print('# Status Code = ', resp.status_code)
        print('# Raw Messages = ', resp.json())
        additions['raw_messages'] = resp.json

    return APIResult(resp.json(), additions)

# Retrieving Content via HTTP GET Request
def send_get_content(url: str) -> ContentsResult:
    print()
    print('# 💾 Retrieve contents')
    print('# url = ', url)

    # Send Request
    resp = requests.get(url)

    # Display request datetime in JST
    # Mon, 06 Feb 2023 02:03:09 GMT
    req_time = datetime.strptime(
        resp.headers['Date'], '%a, %d %b %Y %H:%M:%S %Z').astimezone(JST)
    print('# ⏰ Request datetime = ', req_time,
          int(req_time.timestamp() * 1000))

    # Preparation for results return
    additions = dict()
    additions['status_code'] = resp.status_code
    additions['headers'] = resp.headers

    if resp.status_code != 200:
        print()
        print('# ❌ GET contents failed')
        print('# Status Code = ', resp.status_code)
        print('# Raw Messages = ', resp.json())
        additions['raw_messages'] = resp.json

    return ContentsResult(resp.content, additions)

# Extract filenames from URL strings
def extract_filename_from_URL(url: str) -> str:
    # Extract filenames
    name = urlparse(url).path.rsplit('/', 1)[1]

    if name == None or name == '':
        return ''

    # Decode and re-encode
    return quote(unquote(name))

# Initialize directory
def init_dir(path: str, new: bool = True) -> None:
    print()
    print('# 📦 Initialize directory')
    print('# path = ', path)
    print('# new = ', new)

    # If the directory does not exist, create it.
    os.makedirs(path, exist_ok=True)

    # Get a list of files
    name = '**'
    files = glob.glob(os.path.join(path, name), recursive=True)

    print()
    print('# 🗑️ Delete target = ', files)

    # Delete files
    shutil.rmtree(path)

    # Create New
    if new:
        os.makedirs(path, exist_ok=True)

# ZIP the selected directory and download it locally
def download_dir_locally(root_path: str, out_path: str, init: bool = True) -> None:
    print()
    print('# 🪂 Compress the directory and download it.')
    print('# root_path = ', root_path)
    print('# out_path = ', out_path)

    # Get a list of files
    name = '**'
    file_list = glob.glob(os.path.join(root_path, name), recursive=True)
    print('# files = ', file_list)

    # Initialize output destination directory
    if init:
        init_dir(out_path)

    # ZIP file name
    zip_name = datetime.now(JST).strftime('_%Y%m%d_%H%M%S_archive_')

    # Compress an entire dir
    archive = shutil.make_archive(os.path.join(
        out_path, zip_name), format='zip', root_dir=root_path)
    files.download(archive)

    print()
    print('# 🙏 Please wait while downloading completes.')
    print('# 📚 Download Target = ',  archive)



# API Client Class
# Combine API-related implementations into this class
class APIClient():
    endpoint_url: str
    api_key: str
    token: str

    def __init_vars(self):
        self.endpoint_url = ''
        self.api_key = ''
        self.token = ''

    def __init__(self, endpoint_url: str):
        self.__init_vars()
        self.endpoint_url = endpoint_url

    def __del__(self):
        self.__init_vars()

    def __get_auth_headers(self):
        return {
            'Content-Type': 'application/json',
            'X-Soracom-API-Key': self.api_key,
            'X-Soracom-Token': self.token,
        }

    # /auth
    # Authenticate API access and issue API key and API token
    # https://users.soracom.io/ja-jp/tools/api/reference/#/Auth/auth
    def authenticate_user(self, email: str, password: str, opid: str, name: str, mfa_code: str = None, timeout: int = 3600) -> APIResult:
        api_path = '/auth/'
        url = merge_url(self.endpoint_url, api_path)
        headers = {'Content-Type': 'application/json'}

        # authentication uses the following values
        # Root: email, password
        # SAM: opid, name, password
        data = dict()

        # prepare the value for the Root case
        if email != None and email != '':
            if password != None and password != '':
                data['email'] = email
                data['password'] = password

        # prepare the value for the SAM case
        # If both are valid, SAM is preferred
        if opid != None and opid != '':
            if name != None and name != '':
                if password != None and password != '':
                    data['operatorId'] = opid
                    data['userName'] = name
                    data['password'] = password

        # set the API key expiration time (Default 1 hour)
        data['tokenTimeoutSeconds'] = timeout

        # MFA Authentication Code
        if mfa_code != None and mfa_code != '':
            data['mfaOTPCode'] = mfa_code

        # Send Request
        return send_post_request(url, headers=headers, data=data)

    # /sora_cam/devices
    # Get the list of sora-cam compatible cameras
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDevices

    def list_sora_cam_devices(self) -> APIResult:
        api_path = '/sora_cam/devices'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}
    # Get information about cameras compatible with sora-cam
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDevice

    def get_sora_cam_device(self, id: str) -> APIResult:
        api_path = '/sora_cam/devices/' + id
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/exports/usage
    # Get the number of still images that can be exported from a sora-cam compatible camera and the exportable time of the recorded video
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceExportUsage

    def get_sora_cam_export_usage(self, id: str) -> APIResult:
        api_path = '/sora_cam/devices/' + id + '/exports/usage'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/stream
    # Get information about downloading streaming video (real-time video / recorded video)
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceStreamingVideo

    def get_sora_cam_streaming_video(self, id: str, time_from: int = None, time_to: int = None) -> APIResult:
        api_path = '/sora_cam/devices/' + id + '/stream'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Set parameters as needed.
        params = None
        if time_from != None and time_from > 0:
            if time_to != None and time_to > 0:
                params = {
                    'from': time_from,
                    'to': time_to,
                }

        # Send Request
        return send_get_request(url, headers=headers, params=params)

    # /sora_cam/devices/{device_id}/events
    # Retrieve a list of events from SoraCam compatible cameras.
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceEventsForDevice

    def list_sora_cam_events_for_device(self, id: str, limit: int = None, time_from: int = None, time_to: int = None, last_key: str = None) -> APIResult:
        api_path = '/sora_cam/devices/' + id + '/events'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Set parameters as needed.
        params = dict()

        if limit != None and limit > 0:
            params['limit'] = limit

        if time_from != None and time_from > 0:
            params['from'] = time_from

        if time_to != None and time_to > 0:
            params['to'] = time_to

        if last_key != None and last_key != '':
            params['last_evaluated_key'] = last_key

        if len(params) <=0:
            params = None

        # Send Request
        return send_get_request(url, headers=headers, params=params)

    # /sora_cam/devices/{device_id}/atomcam/settings
    # Get SoraCam compatible camera settings
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/getSoraCamDeviceAtomCamSettings

    def get_sora_cam_atom_cam_settings(self, id: str) -> APIResult:
        api_path = '/sora_cam/devices/' + id + '/atomcam/settings'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Send Request
        return send_get_request(url, headers=headers)

    # /sora_cam/devices/{device_id}/recordings_and_events
    # Get a list of time periods and a list of events recorded by SoraCam compatible camera.
    # https://users.soracom.io/ja-jp/tools/api/reference/#/SoraCam/listSoraCamDeviceRecordingsAndEvents

    def list_sora_cam_recordings_and_events(self, id: str, time_from: int = None, time_to: int = None, last_key: str = None) -> APIResult:
        api_path = '/sora_cam/devices/' + id + '/recordings_and_events'
        url = merge_url(self.endpoint_url, api_path)
        headers = self.__get_auth_headers()

        # Set parameters as needed.
        params = dict()

        if time_from != None and time_from > 0:
            params['from'] = time_from

        if time_to != None and time_to > 0:
            params['to'] = time_to

        if last_key != None and last_key != '':
            params['last_evaluated_key'] = last_key

        if len(params) <=0:
            params = None

        # Send Request
        return send_get_request(url, headers=headers, params=params)



# --------------- Execution ---------------
# Process start message display
print_start_msg()

print()
print('# ℹ️ Python Version = ', sys.version)

# Process end message display
print_end_msg()


In [ ]:
# @title ステップ 3: OpenAI API の API キーを設定する

# --------------- Definition ---------------
# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Login using OpenAI Organization ID.
use_org = False #@param {type:"boolean"}

# @markdown - - -


from openai import OpenAI


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Prepare UI for OpenAI API Keys
# Organization ID input field
input_openai_org = widgets.Text(
    value=None,
    placeholder='Enter Organization ID',
    disabled=False
)

# API Key input field
input_opneai_api_key = widgets.Password(
    value=None,
    placeholder='Enter API Key',
    disabled=False
)

# authentication button
button_auth_openai = widgets.Button(
    description='Apply',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Apply',
    icon='rocket'  # (FontAwesome names without the `fa-` prefix)
)

# UI display
print()
print('➕➕➕ 🔰 Use the OpenAI API. Fill in the required fields. Then press the Apply button. ➕➕➕')
print()

if use_org:
    display(widgets.Label(value="🆔 Organization ID: "))
    display(input_openai_org)

display(widgets.Label(value="🔑 API Key: "))
display(input_opneai_api_key)

print()
display(button_auth_openai)

# Process end message display
print_end_msg()

# OpenAI Client
openai_client = None

# Function called when a button is pressed
def on_button_auth_openai_handler(change):
    api_key = input_opneai_api_key.value
    org_Id = input_openai_org.value

    # Check input values
    if api_key == None or api_key == '':
        print()
        print('➕➕➕ ⛔ OpenAI API key have not been entered. Please check your input. ➕➕➕')
        return

    if use_org:
        if org_Id == None or org_Id == '':
            print()
            print('➕➕➕ ⛔ OpenAI Organization ID have not been entered. Please check your input. ➕➕➕')
            return

    # Set authentication keys OpenAI client
    global openai_client
    openai_client = OpenAI(api_key=api_key, organization=org_Id)

    # Verify if authenticated correctly
    print()
    if use_org:
        print('# 🆔 OpenAI Organization ID = ', openai_client.organization)

    if len(openai_client.models.list().data) <= 0:
        print()
        print('➕➕➕ ⛔ Could not confirm operation with the entered API key. Please check your input. ➕➕➕')
        return

    # Clear input values to prevent reuse
    org_Id = ''
    input_openai_org.value = ''
    api_key = ''
    input_opneai_api_key.value = ''

    print('# 📜 Number of all AI models: ', len(openai_client.models.list().data))
    print()
    print('# 🔑 API access has been verified. 💯')

    # Process end message display
    print_end_msg()


# Handler Setup
button_auth_openai.on_click(on_button_auth_openai_handler)


In [ ]:
# @title ステップ 4: 利用する AI モデルを選択する

# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Get a list of models
print()
print('# 🆕 All AI models: ', openai_client.models.list().data)

# Display only vision capable models.
ai_model_list = list()
for item in openai_client.models.list().data:
    if item.id.startswith('gpt-'):
        if item.id.find('-vision') != -1:
            ai_model_list.append(item.id)
ai_model_list.sort(reverse=True)

print('# 🏞️ Vision models: ', ai_model_list)

# Models Dropdown list
select_ai_models = widgets.Dropdown(
    options=ai_model_list,
    disabled=False,
    value=ai_model_list[0],
)

print()
print('➕➕➕ 🔰 Select from Vision models ➕➕➕')
print()
display(widgets.Label(value="🏞️ Vision models： "))
display(select_ai_models)

# Select the first model in the list
selected_ai_model: str = ai_model_list[0]

print()
print('# ✅ selected AI model = ', selected_ai_model)

# Process end message display
print_end_msg()


# Function called when a Dropdown is selected
def on_select_ai_models_handler(change):
    selected_Item = change['new']

    # Maintain information about selected models
    global selected_ai_model
    selected_ai_model = selected_Item

    print('')
    print('# ✅ selected AI model = ', selected_ai_model)

    # Process end message display
    print_end_msg()


# Handler Setup
select_ai_models.observe(on_select_ai_models_handler, names='value')


In [ ]:
# @title ステップ 5: SORACOM API を使うための認証処理をする

# --------------- Definition ---------------
# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Set the SORACOM API endpoint.
endpoint_url = 'https://api.soracom.io/v1'  # @param {type:"string"}

# @markdown Select user to login to SORACOM User Console.
login_type = "SAM User"  # @param ["Root User", "SAM User"]

# @markdown Login using multi-factor authentication.
use_mfa = False #@param {type:"boolean"}

# @markdown - - -


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# init API client
# We will continue to use this variable when calling the API in the following sections
api_client: APIClient = APIClient(endpoint_url)

# Prepare UI for login
# Root User email address input field
input_email = widgets.Text(
    value=None,
    placeholder='Enter Email Address',
    disabled=False
)

# SAM User operator ID input field
input_operator_Id = widgets.Text(
    value=None,
    placeholder='Enter Operator ID',
    disabled=False
)

# SAM User user name input field
input_user_name = widgets.Text(
    value=None,
    placeholder='Enter User Name',
    disabled=False
)

# User password input field
input_password = widgets.Password(
    value=None,
    placeholder='Enter Password',
    disabled=False
)

# MFA code input field
input_mfa_code = widgets.Text(
    value=None,
    placeholder='Enter MFA Code',
    disabled=False
)

# Login button
login_button = widgets.Button(
    description='Login',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Login',
    icon='user'  # (FontAwesome names without the `fa-` prefix)
)

# UI display by login type
if login_type == 'Root User':
    print()
    print('➕➕➕ 🔰 Login as the Root User. Fill in the required fields. Then press the Login button. ➕➕➕')
    print()

    display(widgets.Label(value="📧 Email Address: "))
    display(input_email)

elif login_type == 'SAM User':
    print()
    print('➕➕➕ 🔰 Login as the SAM User. Fill in the required fields. Then press the Login button. ➕➕➕')
    print()

    display(widgets.Label(value="🪪 Operator ID: "))
    display(input_operator_Id)
    display(widgets.Label(value="📛 User Name: "))
    display(input_user_name)

display(widgets.Label(value="🔑 Password: "))
display(input_password)

if use_mfa:
    display(widgets.Label(value="🔒 MFA Code: "))
    display(input_mfa_code)

print()
display(login_button)

# Process end message display
print_end_msg()


# Function called when a button is pressed
def on_login_button_handler(change):
    operator_Id = input_operator_Id.value
    user_name = input_user_name.value
    password = input_password.value
    email = input_email.value
    mfa_code = None

    # Check input values
    if login_type == 'Root User':
        if email == None or email == '' or password == None or password == '':
            print()
            print(
                '➕➕➕ ⛔ The login information for the Root User is missing. Please check your input. ➕➕➕')
            return
    elif login_type == 'SAM User':
        if operator_Id == None or operator_Id == '' or password == None or password == '' or user_name == None or user_name == '':
            print()
            print(
                '➕➕➕ ⛔ The login information for the SAM User is missing. Please check your input. ➕➕➕')
            return

    # Check MFA Authentication Code
    if use_mfa:
        mfa_code = input_mfa_code.value
        if not (mfa_code.isdecimal() and mfa_code.isascii()):
            print()
            print(
                '➕➕➕ ⛔ Please enter only numbers for the MFA verification code. ➕➕➕')
            return

    # Perform Authentication
    api_result = api_client.authenticate_user(
        email, password, operator_Id, user_name, mfa_code)

    if not api_result.check_success():
        print()
        print('➕➕➕ ⛔ API authentication failed. Please check your input. ➕➕➕')
        return

    auth_resp = api_result.response
    if auth_resp['apiKey'] != None and auth_resp['apiKey'] != '':
        if auth_resp['token'] != None and auth_resp['token'] != '':
            print()
            print('# 🔑 API access has been authenticated 💯')

            # Keep your API keys
            api_client.api_key = auth_resp['apiKey']
            api_client.token = auth_resp['token']

            # Clear input values to prevent reuse
            operator_Id = ''
            input_operator_Id.value = ''
            user_name = ''
            input_user_name.value = ''
            password = ''
            input_password.value = ''
            email = ''
            input_email.value = ''
            mfa_code = ''
            input_mfa_code.value = ''

    # Process end message display
    print_end_msg()


# Handler Setup
login_button.on_click(on_login_button_handler)


In [ ]:
# @title ステップ 6: カメラを 1 台選択する

# --------------- Definition ---------------
# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Display only connectable cameras.
connectable_cameras = True #@param {type:"boolean"}

# @markdown - - -


# Returns an edited list of devices for the dropdown
def create_device_list_for_dropdown(devices: list) -> list:
    result = list()

    if len(devices) <= 0:
        return result

    for item in devices:
        if connectable_cameras:
            if not item['connected']:
                continue

        # The following order according to the user console display
        # name / connected / firmwareVersion / productDisplayName / deviceId
        status = '🌈 Connected' if item['connected'] else '🌀 Disconnected'
        display = '{} / {} / {} / {} / {}'.format(
            item['name'], status, item['firmwareVersion'], item['productDisplayName'], item['deviceId'])
        result.append((display, item))

    return result

# Display remaining time for video export
def display_remaining_export_time(id: str) -> None:
    # Time remaining for video export
    usgae = None
    api_result = api_client.get_sora_cam_export_usage(id)

    if api_result.check_success():
        usgae = api_result.response

    print('')
    print('# 🆓 Exportable Times = ', api_result.get_display_response())
    print('# 🔎 Time remaining of month = ', '{} hours {} minutes {} seconds'.format(int(
        usgae['video']['remainingSeconds'] / 3600), int((usgae['video']['remainingSeconds'] % 3600) / 60), int(usgae['video']['remainingSeconds'] % 60)))


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Get a list of cameras
device_list = None
api_result = api_client.list_sora_cam_devices()

if api_result.check_success():
    device_list = api_result.response

print()
print('# 📷 Camera List = ', device_list)
print('# 📜 Number of All Camera List: ', len(device_list))

# Camera Dropdown list
select_devices = widgets.Dropdown(
    options=create_device_list_for_dropdown(device_list),
    disabled=False,
    value=None,
)

print()
print('➕➕➕ 🔰 Select from Camera List ➕➕➕')
print()
display(widgets.Label(value="📷 Camera List: "))
display(select_devices)

# Select the first camera in the list
device_selected: dict = device_list[0]

# Process end message display
print_end_msg()


# Function called when a Dropdown is selected
def on_select_devices_handler(change):
    selected_Item = change['new']

    # Get details about the selected camera
    device_info = None
    api_result = api_client.get_sora_cam_device(selected_Item['deviceId'])

    if api_result.check_success():
        device_info = api_result.response

    print('')
    print('# 📸 Selected Camera Details = ', api_result.get_display_response())

    # Get ATOM Cam Setting about the selected camera
    api_result = api_client.get_sora_cam_atom_cam_settings(selected_Item['deviceId'])

    print('')
    print('# 📷 Selected Camera ATOM Cam Settings = ', api_result.get_display_response())

    # Display remaining time for video export
    display_remaining_export_time(selected_Item['deviceId'])

    # Maintain information about selected devices
    global device_selected
    device_selected = device_info

    # Process end message display
    print_end_msg()


# Handler Setup
select_devices.observe(on_select_devices_handler, names='value')


In [ ]:
# @title ステップ 7: イベント一覧の取得期間を設定する

# --------------- Definition ---------------

# Define variables to hold values
event_time_info: ExportTimeInfo = None


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Prepare a list since time picker is not available
hours = [i for i in range(24)]
minutes = [i for i in range(60)]
seconds = [i for i in range(60)]
jst_now = datetime.now(JST)
one_hour_ago = jst_now - timedelta(hours=1)

# Start date
event_start_date = widgets.DatePicker(
    description='Start date:',
    value=one_hour_ago.date(),
    disabled=False
)

# Start time
event_strat_hours = widgets.Dropdown(
    options=hours,
    value=one_hour_ago.hour,
    description='Start time:',
    disabled=False,
)

event_strat_minutes = widgets.Dropdown(
    options=minutes,
    value=one_hour_ago.minute,
    disabled=False,
)

event_strat_seconds = widgets.Dropdown(
    options=seconds,
    value=one_hour_ago.second,
    disabled=False,
)

# End date
event_end_date = widgets.DatePicker(
    description='End date:',
    value=jst_now.date(),
    disabled=False
)

# End time
event_end_hours = widgets.Dropdown(
    options=hours,
    value=jst_now.hour,
    description='End time:',
    disabled=False,
)

event_end_minutes = widgets.Dropdown(
    options=minutes,
    value=jst_now.minute,
    disabled=False,
)

event_end_seconds = widgets.Dropdown(
    options=seconds,
    value=jst_now.second,
    disabled=False,
)

event_time_button = widgets.Button(
    description='Time setting',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Time setting',
    icon='clock'  # (FontAwesome names without the `fa-` prefix)
)

event_full_button = widgets.Button(
    description='Full term',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Full term',
    icon='gas-pump'  # (FontAwesome names without the `fa-` prefix)
)

# Display the time setting UI
print()
print('➕➕➕ 🔰  Set the period for retrieving events.  ➕➕➕')
print()
display(widgets.HBox([event_start_date, event_strat_hours, widgets.Label(
    value='：'), event_strat_minutes, widgets.Label(value='：'), event_strat_seconds]))
display(widgets.HBox([event_end_date, event_end_hours, widgets.Label(
    value='：'), event_end_minutes, widgets.Label(value='：'), event_end_seconds]))
print()
display(widgets.HBox(
    [event_time_button, widgets.Label(value='  '), event_full_button]))

# Process end message display
print_end_msg()


# Function called when a button is pressed
def on_event_time_button_handler(change):
    start_date = event_start_date.value
    strat_hour = event_strat_hours.value
    strat_minute = event_strat_minutes.value
    strat_second = event_strat_seconds.value

    # ISO format for start time
    # '2018-12-31T05:00:30.001000+09:00'
    start_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
        start_date, strat_hour, strat_minute, strat_second)
    start_datetime = datetime.fromisoformat(start_datetime_iso)

    end_date = event_end_date.value
    end_hour = event_end_hours.value
    end_minute = event_end_minutes.value
    end_second = event_end_seconds.value

    # ISO format for end time
    # '2018-12-31T05:00:30.001000+09:00'
    end_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
        end_date, end_hour, end_minute, end_second)
    end_datetime = datetime.fromisoformat(end_datetime_iso)

    time_info = ExportTimeInfo(start_datetime, end_datetime)
    print()
    print('# start_datetime = ', time_info.time_from)
    print('# end_datetime = ', time_info.time_to)
    print('# export_range = ', time_info.time_to - time_info.time_from)

    # Check the entered time
    if not time_info.check_time_limits(False):
        return

    # Holds the entered value
    global event_time_info
    event_time_info = copy.deepcopy(time_info)

    print()
    print('# 🗓️ Set the start and end times for events. 🆗')

    # Process end message display
    print_end_msg()


# Function called when a button is pressed
def on_event_full_button_handler(change):
    # It is treated as full time if no time is specified.
    global event_time_info
    event_time_info = ExportTimeInfo(None, None)

    print()
    print('# 🔋 Set the retrieve all events. 🆗')

    # Process end message display
    print_end_msg()


# Handler Setup
event_time_button.on_click(on_event_time_button_handler)

# Handler Setup
event_full_button.on_click(on_event_full_button_handler)


In [ ]:
# @title ステップ 8: 設定した期間のイベントから 1 つ選択する

# --------------- Definition ---------------

# @markdown # Fill in the required information and run.
# @markdown - - -

# @markdown Only display events that are streamable.
streamable_events = False  # @param {type:"boolean"}

# @markdown - - -


# Retrieve a list of events associated with the camera.
def get_event_list(id: str, limit: int = None, time_from: int = None, time_to: int = None, last_key: str = None) -> list:
    result = list()

    api_result = api_client.list_sora_cam_events_for_device(id, limit, time_from, time_to, last_key)

    if api_result.check_success():
      result.extend(api_result.response)

      # Is there more to the list?
      if api_result.additions.get('headers'):
          if api_result.additions.get('headers').get('X-Soracom-Next-Key'):
              next_key = api_result.additions.get('headers').get('X-Soracom-Next-Key')
              print()
              print('# ⏭ next_key = ', next_key)

              result.extend(get_event_list(id, limit, time_from, time_to, next_key))

    return result

# Create a list to display in the dropdown.
def create_event_list_for_dropdown(event_list: list, filter: bool = False) -> list:
    result = list()

    if len(event_list) <= 0:
        return result

    for item in event_list:
        # The following order according to display
        # date / type / status / image / stream
        date_time = datetime.fromtimestamp(item['time'] / 1000, JST)

        event_type = 'other'
        if item.get('eventInfo'):
            if item.get('eventInfo').get('atomEventV1'):
                if item.get('eventInfo').get('atomEventV1').get('type'):
                    event_type = item.get('eventInfo').get('atomEventV1').get('type')

        status = ''
        if item.get('eventInfo'):
            if item.get('eventInfo').get('atomEventV1'):
                if item.get('eventInfo').get('atomEventV1').get('recordingStatus'):
                    status = item.get('eventInfo').get('atomEventV1').get('recordingStatus')
                    if status == 'completed':
                        status = '✅ completed'

        image = 'unavailable'
        if item.get('eventInfo'):
            if item.get('eventInfo').get('atomEventV1'):
                if item.get('eventInfo').get('atomEventV1').get('picture'):
                    image = item.get('eventInfo').get('atomEventV1').get('picture')
                    if image != None and image != '':
                        image = '🖼️ available'

        start = 0
        if item.get('eventInfo'):
            if item.get('eventInfo').get('atomEventV1'):
                if item.get('eventInfo').get('atomEventV1').get('startTime'):
                    start = item.get('eventInfo').get('atomEventV1').get('startTime')

        end = 0
        if item.get('eventInfo'):
            if item.get('eventInfo').get('atomEventV1'):
                if item.get('eventInfo').get('atomEventV1').get('endTime'):
                    end = item.get('eventInfo').get('atomEventV1').get('endTime')

        stream = 'unavailable'
        stream_state = False
        if start > 0 and end > 0:
            stream = '🎥 available'
            stream_state = True

        display = '{} / {} / {} / {} / {}'.format(date_time, event_type, status, image, stream)
        add_state = True

        # Perform filtering?
        if filter:
            add_state = stream_state

        if add_state:
            result.append((display, item))

    return result


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Get a list of events
events_list = get_event_list(id = device_selected['deviceId'], limit = 100, time_from = event_time_info.get_from_ms(), time_to = event_time_info.get_to_ms())

print()
print('# 🎰 event list = ', events_list)
print('# Number of events: ', len(events_list))

# Get a list of events to view
view_event_list = create_event_list_for_dropdown(events_list, streamable_events)
print('# Number of view events: ', len(view_event_list))

# Events Dropdown list
select_events = widgets.Dropdown(
    options=view_event_list,
    disabled=False,
    value=None,
)

print()
print('➕➕➕ 🔰 Select from Event List ➕➕➕')
print()
display(widgets.Label(value="🌈 Event List: "))
display(select_events)

# Select the first camera in the list
selected_event: dict = events_list[0]

# Process end message display
print_end_msg()


# Function called when a Dropdown is selected
def on_select_events_handler(change):
    selected_Item = change['new']

    # Maintain information about selected devices
    global selected_event
    selected_event = selected_Item

    print('')
    print('# ✅ selected event = ', json.dumps(selected_event, indent=4, ensure_ascii=False))

    # Process end message display
    print_end_msg()


# Handler Setup
select_events.observe(on_select_events_handler, names='value')


In [ ]:
# @title ステップ 9: 選択したイベントのストリーミング映像を再生する

# --------------- Definition ---------------

from google.colab.output import eval_js
from IPython.display import HTML

# Display HTML for streaming video viewing
def display_stream_player(url: str, expiry_time: int) -> None:
    video_html = HTML('''
        <style>
            video {
                width: 960px;
                height: 540px;
            }
        </style>
        <div id="url"></div>
        <div id="time"></div>
        <div>
            <video id="videoPlayer" controls></video>
        </div>
        <script src="//cdn.dashjs.org/latest/dash.all.debug.js"></script>
        <script>
          function init_video(url, time){
            var player = dashjs.MediaPlayer().create();
            player.initialize(document.querySelector("#videoPlayer"), url, true);
            document.getElementById("url").innerHTML = '<p> 🌏 url = ' + url + '</p>';
            document.getElementById("time").innerHTML = '<p>⏳ expiry_time = <b>' + time + '</b></p>';
          }
          init_video("%s", "%s")
        </script>
  ''' % (url, expiry_time))

    print()
    display(video_html)


# Load the video
def laod_stream_video(id: str, time_from: int = None, time_to: int = None, init: bool = True) -> None:
    stream_info = None
    api_result = api_client.get_sora_cam_streaming_video(
        id, time_from, time_to)

    if api_result.check_success():
        stream_info = api_result.response

    video_url = stream_info['playList'][0]['url']
    expiry_time = datetime.fromtimestamp(stream_info['expiryTime'] / 1000, JST)

    print()
    print('# 📺 Video Information = ', stream_info)
    print('# video_url = ', video_url)
    print('# expiry_time = ', expiry_time)

    # Display remaining time for video export
    display_remaining_export_time(device_selected['deviceId'])

    if init:
        # Display HTML for streaming video viewing
        display_stream_player(video_url, expiry_time)
    else:
        # Reflects values to the Javascript
        eval_js('init_video("{}", "{}")'.format(video_url, expiry_time))


# --------------- Execution ---------------
# Process start message display
print_start_msg()

print()
print('# Event time = ', datetime.fromtimestamp(selected_event['time'] / 1000, JST))

# Retrieve the start and end times from the selected event.
start = 0
if selected_event.get('eventInfo'):
    if selected_event.get('eventInfo').get('atomEventV1'):
        if selected_event.get('eventInfo').get('atomEventV1').get('startTime'):
            start = selected_event.get('eventInfo').get('atomEventV1').get('startTime')

end = 0
if selected_event.get('eventInfo'):
    if selected_event.get('eventInfo').get('atomEventV1'):
        if selected_event.get('eventInfo').get('atomEventV1').get('endTime'):
            end = selected_event.get('eventInfo').get('atomEventV1').get('endTime')

stream_time_info: ExportTimeInfo = ExportTimeInfo(None, None)
if start > 0 and end > 0:
    stream_time_info = ExportTimeInfo(datetime.fromtimestamp(start / 1000, JST), datetime.fromtimestamp(end / 1000, JST))

    print('# Event start time = ', stream_time_info.time_from)
    print('# Event end time = ', stream_time_info.time_to)
    print('# Event video duration (seconds) = ', stream_time_info.get_export_sec())

# Reload button
stream_reload_button = widgets.Button(
    description='Reload video',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reload video',
    icon='rotate-right'  # (FontAwesome names without the `fa-` prefix)
)

stream_type_label = widgets.Label(value='ℹ️ Real-time video is playing.')

# Real time or no range?
if stream_time_info.valid_vars():
    stream_type_label.value = 'ℹ️ The specified time period is played back.（{} - {}）'.format(
        stream_time_info.time_from, stream_time_info.time_to)

# Initial display of video
laod_stream_video(device_selected['deviceId'], stream_time_info.get_from_ms(
), stream_time_info.get_to_ms(), True)
print()
display(widgets.HBox([stream_reload_button,
        widgets.Label(value='　'), stream_type_label]))

# Process end message display
print_end_msg()

# Function called when a button is pressed
def on_stream_reload_button_handler(change):
    # Video reload process
    laod_stream_video(device_selected['deviceId'], stream_time_info.get_from_ms(
    ), stream_time_info.get_to_ms(), False)

    # Process end message display
    print_end_msg()


# Handler Setup
stream_reload_button.on_click(on_stream_reload_button_handler)


In [ ]:
# @title ステップ 10: 選択したイベントの画像を取得する

# --------------- Definition ---------------

# Define where to store downloaded files
EVENT_IMAGE_DIR: Final = './event_image'


# Download event image.
def download_event_image(url: str, path: str) -> None:
    print()
    print('# 🎨 Download Event Image')
    print('# url = ', url)
    print('# image_path = ', path)

    # Request to retrieve content
    contents_result = send_get_content(url)

    if contents_result.check_success():
        # Save as a image file
        with open(path, "wb") as f:
            f.write(contents_result.content)
            print()
            print('# ✅ The event image has been saved.')

    return None


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Initialize directory to store event image file
init_dir(EVENT_IMAGE_DIR)

# image url
image_url = selected_event.get('eventInfo').get('atomEventV1').get('picture')
image_file_path = os.path.join(EVENT_IMAGE_DIR, extract_filename_from_URL(image_url))

# Download event image.
download_event_image(image_url, image_file_path)

# Process end message display
print_end_msg()

In [ ]:
# @title ステップ 11: 取得したイベント画像を表示する

# --------------- Definition ---------------
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image


# Display images in a grid
def display_grid_images(path: str, name: str, width: int = 15, height: int = 15) -> list:
    # Define number of image columns to display
    DISPLAY_IMAGE_COLUMNS: Final = 3

    print()
    print('# path = ', path)
    print('# name = ', name)

    # Get file list
    file_list = glob.glob(os.path.join(path, name), recursive=True)
    file_list.sort()

    print('# 🗃️ file list = ', file_list)
    print('# file count = ', len(file_list))
    print()

    # Number of columns
    cols = DISPLAY_IMAGE_COLUMNS

    # Number of rows
    counts = len(file_list)
    rows = (counts // cols) if counts % cols == 0 else (counts // cols + 1)

    fig = plt.figure(figsize = (width, height))
    grid = ImageGrid(fig, 111, nrows_ncols = (rows, cols), axes_pad = 0)

    # Set image
    for ax, file in zip(grid, file_list):
        ax.imshow(Image.open(file))

    # Hide axes
    for ax in grid.axes_all:
        ax.axis('off')

    # show images
    plt.show()

    return file_list


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Display images in a grid
display_grid_images(EVENT_IMAGE_DIR, '*.jpg')

# Process end message display
print_end_msg()

In [ ]:
# @title ステップ 12: 取得したイベント画像を OpenAI で解析する

# --------------- Definition ---------------

import base64


# Chat with OpenAI
# https://platform.openai.com/docs/api-reference/chat/create
def create_chat_completion(user_msg: str, model: str, image_path: str = None, temperature: float = None, msg_history: list = None, system_msg: str = None, token_limt: int = 4000) -> list:
    params = dict()
    params['max_tokens'] = token_limt

    if model != None and model != '':
        params['model'] = model

    if temperature != None:
        params['temperature'] = temperature

    messages = list()
    if system_msg != None and system_msg != '':
        messages.append({"role": "system", "content": system_msg})

    # Create a user message
    content_item = {"role": "user", "content": user_msg}
    if image_path != None and image_path != '':
        # Attach image file
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode('utf-8')
            content_item = {"role": "user", "content": [{"type": "text", "text": user_msg},{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}]}

    messages.append(content_item)
    params['messages'] = messages
    if msg_history != None and len(msg_history) > 0:
        params['messages'] = msg_history + messages

    print()
    print('# send params = ', params)

    print()
    print('# 🎨 image = ', image_path)
    print('# 🍺 AI Model = ', model)
    print()
    print('# 👻 System message = ', system_msg)
    print('# 💸 Max tokens = ', token_limt)
    print('# 🌡️ Temperature = ', temperature)
    print()
    print('# 🧑‍💻 User message = ', user_msg)

    # Call OpenAI with configured information
    chat_completion = openai_client.chat.completions.create(**params)

    print()
    print('# chat result = ', chat_completion)
    print()
    msg_history.extend(messages)

    # Confirm reply
    result_msg = list()
    for item in chat_completion.choices:
        print('# 🤖 Reply message = ')
        print(item.message.content.strip())

        # Keep role and content
        msg_history.append({"role": item.message.role, "content": item.message.content})
        result_msg.append({"role": item.message.role, "content": item.message.content})

    print()
    print('# 📦 messages = ', msg_history)
    print('# 💰 usage = ', chat_completion.usage)

    return result_msg


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# User content input field
input_user_content = widgets.Textarea(
    value='',
    placeholder='Enter what you want to ask',
    disabled=False
)

# API temperature select field
slider_api_temperature = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=2.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Max tokens input field
input_token_limit = widgets.IntText(
    value=4000,
    placeholder='Enter a message to the system',
    disabled=False
)

# System content input field
input_system_content = widgets.Textarea(
    value='',
    placeholder='Enter a message to the system',
    disabled=False
)

# send button
send_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Send',
    icon='walkie-talkie'  # (FontAwesome names without the `fa-` prefix)
)

# Display images in a grid
image_list = display_grid_images(EVENT_IMAGE_DIR, '*.jpg', 5, 5)
print()
print('# 🍺 AI Model = ', selected_ai_model)

print()
print('➕➕➕ 🔰 Enter chat message ➕➕➕')
print()

display(widgets.Label(value="👻 System message: "))
display(input_system_content)
display(widgets.Label(value="💸 Max tokens: "))
display(input_token_limit)
display(widgets.Label(value="🌡️ Temperature: "))
display(slider_api_temperature)

print()
display(widgets.Label(value="🧑‍💻 User message:"))
display(input_user_content)

print()
display(send_button)

# Keep a record of your conversations.
user_history: list = list()

# Process end message display
print_end_msg()

# Function called when a button is pressed
def on_send_button_handler(change):
    user_content = input_user_content.value
    system_content = input_system_content.value
    temperature = slider_api_temperature.value
    token_limt = input_token_limit.value

    # Check input values
    if user_content == None or user_content == '':
        print()
        print('➕➕➕ ⛔ No query has been entered. Please check your input. ➕➕➕')
        return

    if token_limt == None or token_limt <= 0 :
        print()
        print('➕➕➕ ⛔ Incorrect entry about Max tokens. Please check your input. ➕➕➕')
        return

    # Attach the image only once at first
    file_path = None
    if len(user_history) <= 0:
        file_path = image_list[0]

    # Call OpenAI with the entered information
    result_msg = create_chat_completion(user_content, selected_ai_model, file_path, temperature, user_history, system_content, token_limt)

    user_content = ''
    input_user_content.value = user_content
    system_content = ''
    input_system_content.value = system_content
    slider_api_temperature.value = temperature
    temperature = None
    input_token_limit.value = token_limt
    token_limt = None

    # Process end message display
    print_end_msg()

# Handler Setup
send_button.on_click(on_send_button_handler)


In [ ]:
# @title ステップ 13: ダウンロード対象のディレクトリを選択する

# --------------- Execution ---------------
# Process start message display
print_start_msg()

USER_CONTENT_DIR: Final = '/content/'

path = USER_CONTENT_DIR
name = '**'

print()
print('# path = ', path)
print('# name = ', name)

# Get Dir list
file_list = glob.glob(os.path.join(path, name), recursive=True)
dir_list = [f for f in file_list if os.path.isdir(f)]
dir_list.sort()

print('# 🗂️ directories = ', dir_list)

# Dir Dropdown list
select_dirs = widgets.Dropdown(
    options=dir_list,
    disabled=False,
    value=None,
)
print()
print('➕➕➕ 🔰 Please select one from the list of directories below. ➕➕➕')
print()

display(widgets.Label(value="🗂️ Directories:"))
display(select_dirs)

# Process end message display
print_end_msg()

# Select the first dir in the list
dir_selected = dir_list[0]

# Function called when a Dropdown is selected
def on_select_dirs_handler(change):
    selected_item = change['new']

    print('')
    print('# 🎯 Selected Dir = ', selected_item)

    # Get a list of files in a dir
    files = glob.glob(os.path.join(selected_item, '*'))
    print('# 📄 File List = ', files)

    if selected_item == USER_CONTENT_DIR:
      print()
      print('# 🧨 The root of the content folder is specified. Compression and download will take some time. Please check before executing.')

    global dir_selected
    dir_selected = selected_item

    # Process end message display
    print_end_msg()


# Handler Setup
select_dirs.observe(on_select_dirs_handler, names='value')



In [ ]:
# @title ステップ 14: 選択したディレクトリをローカルにダウンロードする

# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Download the whole specified directory
download_dir_locally(dir_selected, ZIP_WORK_DIR)

# Process end message display
print_end_msg()


In [ ]:
# @title (Tool) 1: 録画状態の取得期間を設定する

# --------------- Definition ---------------

# Define variables to hold values
data_time_info: ExportTimeInfo = None

# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Prepare a list since time picker is not available
hours = [i for i in range(24)]
minutes = [i for i in range(60)]
seconds = [i for i in range(60)]
jst_now = datetime.now(JST)
one_hour_ago = jst_now - timedelta(hours=1)

# Start date
data_start_date = widgets.DatePicker(
    description='Start date:',
    value=one_hour_ago.date(),
    disabled=False
)

# Start time
data_strat_hours = widgets.Dropdown(
    options=hours,
    value=one_hour_ago.hour,
    description='Start time:',
    disabled=False,
)

data_strat_minutes = widgets.Dropdown(
    options=minutes,
    value=one_hour_ago.minute,
    disabled=False,
)

data_strat_seconds = widgets.Dropdown(
    options=seconds,
    value=one_hour_ago.second,
    disabled=False,
)

# End date
data_end_date = widgets.DatePicker(
    description='End date:',
    value=jst_now.date(),
    disabled=False
)

# End time
data_end_hours = widgets.Dropdown(
    options=hours,
    value=jst_now.hour,
    description='End time:',
    disabled=False,
)

data_end_minutes = widgets.Dropdown(
    options=minutes,
    value=jst_now.minute,
    disabled=False,
)

data_end_seconds = widgets.Dropdown(
    options=seconds,
    value=jst_now.second,
    disabled=False,
)

data_time_button = widgets.Button(
    description='Time setting',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Time setting',
    icon='clock'  # (FontAwesome names without the `fa-` prefix)
)

data_full_button = widgets.Button(
    description='Full term',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Full term',
    icon='gas-pump'  # (FontAwesome names without the `fa-` prefix)
)

# Display the time setting UI
print()
print('➕➕➕ 🔰 Set the period for retrieving data. ➕➕➕')
print()
display(widgets.HBox([data_start_date, data_strat_hours, widgets.Label(
    value='：'), data_strat_minutes, widgets.Label(value='：'), data_strat_seconds]))
display(widgets.HBox([data_end_date, data_end_hours, widgets.Label(
    value='：'), data_end_minutes, widgets.Label(value='：'), data_end_seconds]))
print()
display(widgets.HBox(
    [data_time_button, widgets.Label(value='  '), data_full_button]))

# Process end message display
print_end_msg()


# Function called when a button is pressed
def on_data_time_button_handler(change):
    start_date = data_start_date.value
    strat_hour = data_strat_hours.value
    strat_minute = data_strat_minutes.value
    strat_second = data_strat_seconds.value

    # ISO format for start time
    # '2018-12-31T05:00:30.001000+09:00'
    start_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
        start_date, strat_hour, strat_minute, strat_second)
    start_datetime = datetime.fromisoformat(start_datetime_iso)

    end_date = data_end_date.value
    end_hour = data_end_hours.value
    end_minute = data_end_minutes.value
    end_second = data_end_seconds.value

    # ISO format for end time
    # '2018-12-31T05:00:30.001000+09:00'
    end_datetime_iso = '{}T{:0=2}:{:0=2}:{:0=2}.000000+09:00'.format(
        end_date, end_hour, end_minute, end_second)
    end_datetime = datetime.fromisoformat(end_datetime_iso)

    time_info = ExportTimeInfo(start_datetime, end_datetime)
    print()
    print('# start_datetime = ', time_info.time_from)
    print('# end_datetime = ', time_info.time_to)
    print('# export_range = ', time_info.time_to - time_info.time_from)

    # Check the entered time
    if not time_info.check_time_limits(False):
        return

    # Holds the entered value
    global data_time_info
    data_time_info = copy.deepcopy(time_info)

    print()
    print('# 🗓️ Set the start and end times for data. 🆗')

    # Process end message display
    print_end_msg()


# Function called when a button is pressed
def on_data_full_button_handler(change):
    # It is treated as full time if no time is specified.
    global data_time_info
    data_time_info = ExportTimeInfo(None, None)

    print()
    print('# 🔋 Set the retrieve all data. 🆗')

    # Process end message display
    print_end_msg()


# Handler Setup
data_time_button.on_click(on_data_time_button_handler)

# Handler Setup
data_full_button.on_click(on_data_full_button_handler)


In [ ]:
# @title (Tool) 2: 設定した期間の録画状態を取得する

# --------------- Definition ---------------

# Retrieve a list of recording associated with the camera.
def get_recording_list(id: str, time_from: int = None, time_to: int = None, last_key: str = None) -> (list, list):
    result_records = list()
    result_events = list()

    api_result = api_client.list_sora_cam_recordings_and_events(id, time_from, time_to, last_key)

    if api_result.check_success():
      # HTTP response is 200, but body may be missing
      if api_result.response != None:
          if api_result.response.get('records'):
              result_records.extend(api_result.response.get('records'))
          if api_result.response.get('events'):
              result_events.extend(api_result.response.get('events'))

      # Is there more to the list?
      if api_result.additions.get('headers'):
          if api_result.additions.get('headers').get('X-Soracom-Next-Key'):
              next_key = api_result.additions.get('headers').get('X-Soracom-Next-Key')
              print()
              print('# ⏭ next_key = ', next_key)

              record_list, event_list = get_recording_list(id, time_from, time_to, next_key)
              result_records.extend(record_list)
              result_events.extend(event_list)

    return result_records, result_events


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Check the recording status
record_list, event_list = get_recording_list(id = device_selected['deviceId'], time_from = data_time_info.get_from_ms(), time_to = data_time_info.get_to_ms())

print()
print('# 📼 record list = ', record_list)
print('# 📜 Number of All records list: ', len(record_list))

edited_records = list()
for item in record_list:
    item_val = None
    item_dict = dict()

    if item.get('startTime'):
        item_val = item.get('startTime')
        date_time = datetime.fromtimestamp(item_val / 1000, JST)
        date_time = date_time.replace(tzinfo=None).replace(microsecond=0)
        item_dict['startTime'] = date_time

    if item.get('endTime'):
        item_val = item.get('endTime')
        date_time = datetime.fromtimestamp(item_val / 1000, JST)
        date_time = date_time.replace(tzinfo=None).replace(microsecond=0)
        item_dict['endTime'] = date_time

    if not item_dict.get('endTime'):
        if item_dict.get('startTime'):
            item_dict['endTime'] = item_dict.get('startTime')

    edited_records.append(item_dict)

print('# edited records =', edited_records)

print()
print('# 🎰 event list = ', event_list)
print('# 📜 Number of All events list: ', len(event_list))

edited_events = list()
for item in event_list:
    item_val = None
    item_dict = dict()

    if item.get('startTime'):
        item_val = item.get('startTime')
        date_time = datetime.fromtimestamp(item_val / 1000, JST)
        date_time = date_time.replace(tzinfo=None).replace(microsecond=0)
        item_dict['startTime'] = date_time

    if item.get('endTime'):
        item_val = item.get('endTime')
        date_time = datetime.fromtimestamp(item_val / 1000, JST)
        date_time = date_time.replace(tzinfo=None).replace(microsecond=0)
        item_dict['endTime'] = date_time

    item_dict['type'] = item.get('type')

    edited_events.append(item_dict)

print('# edited events =', edited_events)

# Process end message display
print_end_msg()


In [ ]:
# @title (Tool) 3: 取得した録画状態を表示する

# --------------- Definition ---------------

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import pandas as pd


# Select the appropriate locator and formatter
def select_locator_and_formatter(df: pd.DataFrame) -> (ticker.Locator, str):
    # Analyze date range of data frame
    date_range = df['endTime'].max() - df['startTime'].min()

    # Select the appropriate Locator based on the range
    range_minutes = date_range.total_seconds() / 60
    locator = mdates.MinuteLocator(interval=1)
    formatter = '%y-%m-%d %H:%M'

    # Set no more than 60 to appear on the X axis
    if range_minutes >= 60:
        # Adjust to a range of 60
        locator = mdates.MinuteLocator(interval=int(range_minutes/60))

    # Adjust range in seconds if less than 1 minute
    if range_minutes <= 0:
        range_seconds = date_range.total_seconds()
        locator = mdates.SecondLocator(interval=1)
        formatter = '%y-%m-%d %H:%M:%S'

        if range_seconds >= 60:
            locator = mdates.SecondLocator(interval=int(range_seconds/60))

    return locator, formatter


# --------------- Execution ---------------
# Process start message display
print_start_msg()

# Process records data
df_records = pd.DataFrame(edited_records)

# Create a figure for records data
fig, ax = plt.subplots(figsize=(15, 5))

# Plot records data
for index, row in df_records.iterrows():
    ax.plot([row['startTime'], row['endTime']], [0, 0], marker='o')

locator, formatter = select_locator_and_formatter(df_records)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(mdates.DateFormatter(formatter))
plt.xticks(rotation=45)
plt.xlabel('Time')
plt.ylabel('Records')
plt.title('Timeline of Records')
plt.tight_layout()


# Process events data
df_events = pd.DataFrame(edited_events)

# Create a figure for events data
fig, ax = plt.subplots(figsize=(15, 5))

# Dictionary for event types and their colors
color_dict = {'motion': 'blue'}

# Plot events data
for index, row in df_events.iterrows():
    color = color_dict.get(row['type'], 'red')
    ax.plot([row['startTime'], row['endTime']], [0, 0], marker='x', color=color, label=row['type'])

locator, formatter = select_locator_and_formatter(df_events)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(mdates.DateFormatter(formatter))
plt.xticks(rotation=45)
plt.xlabel('Time')
plt.ylabel('Events')
plt.title('Timeline of Events')
plt.tight_layout()

# Optional legend for events
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

# Show the plots
plt.show()

# Process end message display
print_end_msg()
